## Importing libraries

In [122]:
import random
import numpy as np
import librosa
import pandas as pd
import time
from sklearn.preprocessing import MinMaxScaler

## Looking at the dataset

In [86]:
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6378137 # Radius of Earth in meters
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)

    a = np.sin(dphi/2)**2 + np.cos(phi1)*np.cos(phi2)*np.sin(dlambda/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    return R * c  # Distance in meters

The dataset consists of coordinates and noise levels collected using a recording device around the IISER Bhopal campus. It can be represented as a graph, where locations act as nodes and the distances between them as edges. A new column, `Audio_Path` is created to keep track of the audio files associated with each edge.

In [87]:
# reading the coordinates and distance data
coordinates = pd.read_csv("dataset/coords.csv")

dist = pd.read_csv('dataset/distances.csv')

# creating an audio column which stores the name of audio file corresponfing
audio = []
for i in range(len(dist)):
    audio.append('Audio_'+str(i))

dist['Audio_Path'] = audio

In [88]:
coordinates

,Name,lat,long
0,MP Family Restaurant,23.278627,77.275677
1,IISERB Curb,23.279795,77.277583
2,Bharat Tea Stall,23.279671,77.277671
3,Dairy,23.280711,77.278769
4,Rahul Auto Service,23.280808,77.278596
5,Main Gate,23.280231,77.277188
6,Resident Area Jn,23.280810,77.276617
7,Aadharshila School,23.281609,77.274809
8,Director Bunglow,23.284421,77.274873
9,Shopping Center,23.285449,77.274541


In [89]:
dist

,Source,Destination,Path(m),Audio_Path
0,MP Family Restaurant,IISERB Curb,260,Audio_0
1,IISERB Curb,Rahul Auto Service,160,Audio_1
2,IISERB Curb,Bharat Tea Stall,16,Audio_2
3,IISERB Curb,Main Gate,63,Audio_3
4,MP Family Restaurant,Bharat Tea Stall,260,Audio_4
5,Bharat Tea Stall,Dairy,170,Audio_5
6,Main Gate,Resident Area Jn,83,Audio_6
7,Resident Area Jn,Aadharshila School,280,Audio_7
8,Aadharshila School,Director Bunglow,300,Audio_8
9,Resident Area Jn,VH Jn,400,Audio_9


## Preprocessing the audio data

The STFT (Short-Time Fourier Transform), computed using the `librosa` library, splits the audio into short overlapping windows and analyzes the frequency content in each, allowing us to measure how spectrally "busy" or active the sound is over time.

In [96]:
noise_level = []
for i in range(len(dist)):
    path = 'dataset/'
    file_name =  dist['Audio_Path'][i] 
    try:
        audio_file = path+file_name + '.WAV'
        audio_data, sr = librosa.load(audio_file)
    except:
        audio_file = path+file_name+ '.wav'
        audio_data, sr = librosa.load(audio_file)
    
    audio_amplitude_data = np.abs(librosa.stft(audio_data)).mean()
    noise_level.append(audio_amplitude_data)

In [97]:
dist['Noise_level'] = noise_level
dist

,Source,Destination,Path(m),Audio_Path,RMS,Noise_level
0,MP Family Restaurant,IISERB Curb,260,Audio_0,0.037114,0.131777
1,IISERB Curb,Rahul Auto Service,160,Audio_1,0.034473,0.115371
2,IISERB Curb,Bharat Tea Stall,16,Audio_2,0.039541,0.115244
3,IISERB Curb,Main Gate,63,Audio_3,0.021269,0.088217
4,MP Family Restaurant,Bharat Tea Stall,260,Audio_4,0.037260,0.161299
5,Bharat Tea Stall,Dairy,170,Audio_5,0.027510,0.233734
6,Main Gate,Resident Area Jn,83,Audio_6,0.166985,0.370986
7,Resident Area Jn,Aadharshila School,280,Audio_7,0.032173,0.110173
8,Aadharshila School,Director Bunglow,300,Audio_8,0.040651,0.134968
9,Resident Area Jn,VH Jn,400,Audio_9,0.069095,0.273627


In [124]:
##normalizing the distance and noise level
scaler = MinMaxScaler()
dist["distance_normalized"] = scaler.fit_transform(dist[["Path(m)"]])
dist["noise_level_normalized"] = scaler.fit_transform(dist[["Noise_level"]])
dist

,Source,Destination,Path(m),Audio_Path,RMS,Noise_level,distance_normalized,noise_level_normalized
0,MP Family Restaurant,IISERB Curb,260,Audio_0,0.037114,0.131777,0.635417,0.114921
1,IISERB Curb,Rahul Auto Service,160,Audio_1,0.034473,0.115371,0.375000,0.071639
2,IISERB Curb,Bharat Tea Stall,16,Audio_2,0.039541,0.115244,0.000000,0.071303
3,IISERB Curb,Main Gate,63,Audio_3,0.021269,0.088217,0.122396,0.000000
4,MP Family Restaurant,Bharat Tea Stall,260,Audio_4,0.037260,0.161299,0.635417,0.192808
5,Bharat Tea Stall,Dairy,170,Audio_5,0.027510,0.233734,0.401042,0.383907
6,Main Gate,Resident Area Jn,83,Audio_6,0.166985,0.370986,0.174479,0.746007
7,Resident Area Jn,Aadharshila School,280,Audio_7,0.032173,0.110173,0.687500,0.057925
8,Aadharshila School,Director Bunglow,300,Audio_8,0.040651,0.134968,0.739583,0.123339
9,Resident Area Jn,VH Jn,400,Audio_9,0.069095,0.273627,1.000000,0.489151


## Defining the graph data structure

In [98]:
# This class represent a graph
class Graph:
    # Initialize the class
    def __init__(self, graph_dict=None, directed=True):
        self.graph_dict = graph_dict or {}
        self.directed = directed
        if not directed:
            self.make_undirected()
    # Create an undirected graph by adding symmetric edges
    def make_undirected(self):
        for i in list(self.graph_dict.keys()):
            for (j, dist) in self.graph_dict[i].items():
                self.graph_dict.setdefault(j, {})[i] = dist
    def connect(self, A, B, distance=1):
        self.graph_dict.setdefault(A, {})[B] = distance
        if not self.directed:
            self.graph_dict.setdefault(B, {})[A] = distance
    # Get neighbors or a neighbor
    def get(self, i, j=None):
        links = self.graph_dict.setdefault(i, {})
        if j is None:
            return links
        else:
            return links.get(j)
    # Return a list of nodes in the graph
    def nodes(self):
        S1 = set([k for k in self.graph_dict.keys()])
        S2 = set([k2 for v in self.graph_dict.values() for k2, v2 in v.items()])
        nodes = S1.union(S2)
        return list(nodes)

# This class represent a node
class Node:
    # Initialize the class
    def __init__(self, name:str, parent:str):
        self.name = name
        self.parent = parent
        self.g = 0 # Distance to start node
        self.h = 0 # Distance to goal node
        self.f = 0 # Total cost
    # Compare nodes
    def __eq__(self, other):
        return self.name == other.name
    # Sort nodes
    def __lt__(self, other):
         return self.f < other.f
    # Print node
    def __repr__(self):
        return ('({0},{1})'.format(self.name, self.f))

## A* search algorithm

A* is a pathfinding algorithm that finds the shortest route from a start node to a goal node, combining the strengths of Dijkstra’s algorithm and Greedy Best-First Search. It uses a cost function:

$$
    f(n)=g(n) + h(n)
$$

where $g(n)$ describes actual cost from start to node $n$ and $h(n)$ describes heuristic estimate from node $n$ to the goal. 

Unlike Dijkstra’s algorithm, which uses only $g(n)$, A* also uses a heuristic $h(n)$ to guide the search, making it typically faster and more goal-directed.

In [118]:
def astar_search(graph, heuristics, start, end):
    # Create lists for unexplored nodes and explored nodes
    unexplored = []
    explored = []

    # Create a start node and an goal node
    start_node = Node(start, None)
    goal_node = Node(end, None)

    # Add the start node
    unexplored.append(start_node)
    
    # Loop until the unexplored list is empty
    while len(unexplored) > 0:
        # Sort the unexplored list to get the node with the lowest cost first
        unexplored.sort()
        current_node = unexplored.pop(0)
        
        # Add the current node to the explored list
        explored.append(current_node)
        
        # Check if we have reached the goal, return the path
        if current_node == goal_node:
            path = []
            while current_node != start_node:
                path.append(current_node.name) #+ str(current_node.g)
                current_node = current_node.parent
            path.append(start_node.name)    #+ str(start_node.g)
            # Return reversed path
            return path[::-1]
        
        # Get neighbours
        neighbors = graph.get(current_node.name)
        # Loop neighbors
        for key, value in neighbors.items():
            # Create a neighbor node with given key
            neighbor = Node(key, current_node)
            # Check if that neighbor is in the explored list
            if(neighbor in explored):
                continue

            # Calculate full path cost including the heuristic
            neighbor.g = current_node.g + graph.get(current_node.name, neighbor.name)
            neighbor.h = heuristics.get(neighbor.name)
            neighbor.f = neighbor.g + neighbor.h
            
            # Check if neighbor is in unexplored list and if it has a lower f value
            if(add_to_unexplored(unexplored, neighbor) == True):
                # Everything is green, add neighbor to unexplored list
                unexplored.append(neighbor)

    # Return None, no path is found
    return None

# Check if a neighbor should be added to unexplored list
def add_to_unexplored(unexplored, neighbor):
    for node in unexplored:
        if (neighbor == node and neighbor.f > node.f):
            return False
    return True

## A

In [119]:
# Create a graph
graph = Graph()

for i in  range(len(dist)):
    source = dist["Source"][i]
    destination = dist["Destination"][i]
    distance = dist["Path(m)"][i]
    graph.connect(source, destination, distance)

# Make graph undirected, create symmetric connections
graph.make_undirected()

In [116]:
heuristics = {}

for i in range(len(coordinates)):
    location = coordinates["Name"][i]
    heuristics[location] = 1

In [120]:
path = astar_search(graph, heuristics, 'H7', 'Main Gate')
path

['H7',
 'H7 Jn',
 'H1 Jn',
 'IWD Jn',
 'CCD',
 'Library',
 'Shopping Center',
 'Director Bunglow',
 'VH Jn',
 'Resident Area Jn',
 'Main Gate']